<a href="https://colab.research.google.com/github/jdasam/aat3020/blob/main/notebooks/1_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import string


In [2]:
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"


--2024-03-19 06:24:03--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439742 (429K) [text/plain]
Saving to: ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’

J. K. Rowling - Har 100%[===================>] 429.44K  --.-KB/s    in 0.04s   

2024-03-19 06:24:03 (11.6 MB/s) - ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’ saved [439742/439742]



In [3]:
def remove_punctuation(x):
  return x.translate(''.maketrans('', '', string.punctuation))

def make_tokenized_corpus(corpus):
  out= [ [y.lower() for y in remove_punctuation(sentence).split(' ') if y] for sentence in corpus]
  return [x for x in out if x!=[]]


In [4]:
with open("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", 'r') as f:
  strings = f.readlines()
sample_text = "".join(strings).replace('\n', ' ').replace('Mr.', 'mr').replace('Mrs.', 'mrs').split('. ')


In [5]:
corpus = make_tokenized_corpus(sample_text)
print(corpus[1]) # Corpus is a list of list of strings (words)

['they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense']


In [6]:
from copy import copy
copy(corpus[1])

['they',
 'were',
 'the',
 'last',
 'people',
 'youd',
 'expect',
 'to',
 'be',
 'involved',
 'in',
 'anything',
 'strange',
 'or',
 'mysterious',
 'because',
 'they',
 'just',
 'didnt',
 'hold',
 'with',
 'such',
 'nonsense']

In [7]:
from tqdm import tqdm

sample_sentence = ['they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense']

word_idx = 7
window_size = 2
selected_center_word = sample_sentence[word_idx]

word_pairs = []

for sample_sentence in tqdm(corpus): # tqdm shows the progress bar of for loop
  for word_idx, word in enumerate(sample_sentence):
    for context_idx in range(-window_size, window_size+1):
      if context_idx == 0: continue
      context_word_idx = word_idx + context_idx
      if context_word_idx >= len(sample_sentence) or context_word_idx < 0: continue
      # print(context_idx, context_word_idx, sample_sentence[word_idx], sample_sentence[context_word_idx])
      word_pair = [word, sample_sentence[context_word_idx]]
      word_pairs.append(word_pair)
len(word_pairs)

100%|██████████| 4682/4682 [00:00<00:00, 6980.62it/s]


282372

In [8]:
word_pairs[1000]

['mixing', 'dudley']

In [9]:
# we have to make vocabulary

total_words = []
for sentence in corpus:
  # print(sentence)
  for word in sentence:
    total_words.append(word)

print(total_words[:100])
# for a in corpus:
#   for b in a:
#     c.append(b)
# always use proper variable name

['harry', 'potter', 'and', 'the', 'sorcerers', 'stone', 'chapter', 'one', 'the', 'boy', 'who', 'lived', 'mr', 'and', 'mrs', 'dursley', 'of', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', 'thank', 'you', 'very', 'much', 'they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense', 'mr', 'dursley', 'was', 'the', 'director', 'of', 'a', 'firm', 'called', 'grunnings', 'which', 'made', 'drills', 'he', 'was', 'a', 'big', 'beefy', 'man', 'with', 'hardly', 'any', 'neck', 'although', 'he', 'did', 'have', 'a', 'very', 'large', 'mustache', 'mrs', 'dursley', 'was', 'thin', 'and', 'blonde', 'and', 'had', 'nearly', 'twice', 'the', 'usual']


In [10]:
# we have to get the "unique" item among total words
vocab_set = set(total_words)
print(len(total_words), len(vocab_set))
print(vocab_set)

77597 6038
{'forget', 'hammering', 'scuffles', 'shaves', 'slamming', 'patchwork', 'alchemist', 'blundering', 'elephant', 'flashing', 'pinched', 'pleaded', 'glimmered', 'telephone', 'grinned', 'onward', 'refused', 'spose', 'planets', 'contrary', 'stammered', 'dangers', 'midst', 'paws', 'glancing', 'pocketed', 'sightings', 'build', 'mail', 'jewelbright', 'runnin', 'tuesday', 'latest', 'dressed', 'steal', 'swish', 'stopped', 'knee', 'swallowed', 'grubby', 'core', 'slight', 'stoat', 'cursed', 'blurted', 'gryffindors', 'lemon', 'sped', 'gloomily', 'sacrifices', 'immediately', 'annoying', 'startofterm', 'frantically', 'dive', 'empty', 'selfish', 'circus', 'difference', 'hopefully', 'winged', 'yes', 'anytime', 'creepers', 'floated', 'toothless', 'warn', 'excited', 'surrounded', 'crowded', 'rumbled', 'baseball', 'tunnel', 'weigh', 'gar', 'returned', 'lisa', 'jealous', 'clue', 'four', 'moment', 'ttell', 'rippled', 'hunger', 'tearing', 'surprise', 'tent', 'slumped', 'waste', 'both', 'finish', 't

In [11]:
# this will make error
# vocab_set[0] # set is not subscriptable because it has no order

vocab = list(vocab_set) # converting set into list, so that we can order the items
# sort the liset
vocab = sorted(vocab)
# vocab.sort()
print(vocab)


['\the', '0', '1', '1473', '1637', '17', '1709', '1945', '2', '3', '31', '382', '4', '90', 'a', 'aaaaaaaaaargh', 'aaaargh', 'aaah', 'aargh', 'aback', 'abbott', 'able', 'abnormal', 'aboard', 'abou', 'about', 'aboutface', 'above', 'absolutely', 'absurd', 'abysmal', 'accept', 'accepted', 'accident', 'accidentally', 'accidents', 'according', 'account', 'accountant', 'ache', 'achieve', 'acid', 'aconite', 'acreeping', 'across', 'act', 'acted', 'acting', 'actually', 'adalbert', 'add', 'added', 'address', 'addressed', 'admirable', 'admirals', 'admired', 'admirers', 'admiring', 'admitted', 'adrian', 'advanced', 'advancing', 'adventure', 'advice', 'advised', 'aff', 'affairs', 'affect', 'afford', 'afraid', 'africa', 'african', 'after', 'afternoon', 'afternoons', 'afterward', 'again', 'against', 'age', 'ages', 'ago', 'agony', 'agree', 'agreed', 'agrippa', 'ah', 'aha', 'ahead', 'ahem', 'ahern', 'air', 'airily', 'airplane', 'ajar', 'alarm', 'alarming', 'alas', 'alberic', 'albus', 'alchemist', 'alche

In [12]:
# how can we filter the vocab by its frequency?
from collections import Counter
word_counter = Counter(total_words)
# word_counter['hundreds']
len(word_counter)

# you can use word counter as dictionary
# In python dictionary, dict.keys() gives keys, and dict.values() give values,
# dict.items() give (key, value)
filtered_vocab = []
minimum_occur = 5
for key, value in word_counter.items():
  if value >= minimum_occur:
    filtered_vocab.append(key)
print(len(filtered_vocab))
filtered_vocab.sort() # you must not declare filtered_vocab = filtered_vocab.sort()

1701


In [13]:
print(filtered_vocab)

['a', 'able', 'abou', 'about', 'above', 'across', 'added', 'adventure', 'afford', 'afraid', 'after', 'afternoon', 'again', 'against', 'age', 'ages', 'ago', 'agreed', 'ah', 'ahead', 'air', 'albus', 'alive', 'all', 'alley', 'allowed', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'an', 'and', 'angrily', 'angry', 'animals', 'another', 'answer', 'anxiously', 'any', 'anymore', 'anyone', 'anythin', 'anything', 'anyway', 'anywhere', 'apart', 'appeared', 'archway', 'are', 'arent', 'arm', 'armor', 'arms', 'around', 'arrived', 'arts', 'as', 'ask', 'asked', 'asking', 'asleep', 'at', 'attention', 'aunt', 'awake', 'away', 'baby', 'back', 'backward', 'bacon', 'bad', 'badly', 'bag', 'bald', 'ball', 'balls', 'bane', 'barely', 'barked', 'baron', 'barrier', 'bathroom', 'be', 'beaming', 'beans', 'bear', 'beard', 'became', 'because', 'become', 'becoming', 'bed', 'bedroom', 'been', 'before', 'began', 'behind', 'being', 'believe', 'believed', 'bell', 'bellowed', 'below', 'beneat

In [14]:
# Filter the word_pairs using the vocab
# word_pairs, filtered_vocab
# word_pairs is a list of [word_a, word_b]

filtered_pairs = []
for pair in tqdm(word_pairs):
  word_a, word_b = pair
  # word_a = pair[0]
  # word_b = pair[1]
  # we will include this pair only if both words are in the vocab
  if word_a in filtered_vocab and word_b in filtered_vocab:
    filtered_pairs.append(pair)



100%|██████████| 282372/282372 [00:10<00:00, 28066.35it/s]


In [15]:
# implement same algorithm with list comprehension
filtered_pairs = [pair for pair in word_pairs
                  if pair[0] in filtered_vocab and pair[1] in filtered_vocab]

In [16]:
len(filtered_pairs), len(word_pairs)

(233046, 282372)

In [17]:
filtered_pairs[1000]

['happily', 'she']

In [18]:
# convert word into index of vocab
filtered_vocab.index('happily')

index_pairs = []
for word_a, word_b in tqdm(filtered_pairs):
  word_a_idx = filtered_vocab.index(word_a)
  word_b_idx = filtered_vocab.index(word_b)
  index_pairs.append([word_a_idx, word_b_idx])
len(index_pairs)

100%|██████████| 233046/233046 [00:07<00:00, 29411.93it/s]


233046

In [19]:
filtered_vocab[100]

'behind'

In [20]:
# we can make it faster
# use dictionary to find the index of string

# adict['behind'] = 100
# tok2idx = {}
# for idx, word in enumerate(filtered_vocab):
#   tok2idx[word] = idx # word becomes key, idx becomes value

tok2idx = {word: idx for idx, word in enumerate(filtered_vocab)}

tok2idx['harry']

613

In [21]:
index_pairs = []
for word_a, word_b in tqdm(filtered_pairs):
  word_a_idx = tok2idx[word_a]
  word_b_idx = tok2idx[word_b]
  index_pairs.append([word_a_idx, word_b_idx])


100%|██████████| 233046/233046 [00:00<00:00, 310628.17it/s]


In [22]:
# Why we don't need idx2tok?

def convert_idx_pairs_to_string(pair):
  # pair: [int, int]
  idxa, idxb= pair
  return filtered_vocab[idxa], filtered_vocab[idxb]
convert_idx_pairs_to_string(index_pairs[0])

('harry', 'potter')

In [23]:
# we have to make random vectors for each word in the vocab
# we also have to decide the dimension of the vector

dim = 8
vocab_size = len(filtered_vocab)

word_vectors = torch.randn(vocab_size, dim) / 4
word_vectors, word_vectors.shape

(tensor([[ 0.2593,  0.0433, -0.3650,  ..., -0.5788, -0.5887,  0.5021],
         [-0.2646,  0.1540, -0.1650,  ..., -0.2868, -0.4922, -0.1983],
         [-0.1987, -0.4532,  0.4043,  ...,  0.1768, -0.2591,  0.0825],
         ...,
         [ 0.1410, -0.1560,  0.1037,  ..., -0.0827,  0.3122, -0.1455],
         [ 0.0421, -0.0257,  0.3115,  ...,  0.0695, -0.1727, -0.0372],
         [-0.5766,  0.0908,  0.0406,  ..., -0.2683,  0.1780, -0.5075]]),
 torch.Size([1701, 8]))

In [24]:
# what is the vector for harry?
idx_of_harry = tok2idx['harry']
print(idx_of_harry)
vector_of_harry = word_vectors[idx_of_harry]
vector_of_harry

613


tensor([-0.1886, -0.3664,  0.4148,  0.0823,  0.0359,  0.0883, -0.3901,  0.1089])

In [25]:
filtered_pairs[0], index_pairs[0]

(['harry', 'potter'], [613, 1085])

In [27]:
torch.set_printoptions(sci_mode=False)
vector_of_potter = word_vectors[tok2idx['potter']]

print(vector_of_harry)
print(vector_of_potter)
print(vector_of_harry * vector_of_potter)
print(sum(vector_of_harry * vector_of_potter))

tensor([-0.1886, -0.3664,  0.4148,  0.0823,  0.0359,  0.0883, -0.3901,  0.1089])
tensor([-0.0130, -0.4857, -0.3152,  0.1525,  0.1740,  0.0398,  0.1330, -0.2225])
tensor([ 0.0024,  0.1780, -0.1307,  0.0126,  0.0062,  0.0035, -0.0519, -0.0242])
tensor(-0.0041)


## Dot Product
- Assume we have two vectors $a$ and $b$.
  - $a = [a_1, a_2, a_3, a_4, ..., a_n]$
  - $b = [b_1, b_2, b_3, b_4, ..., b_n]$
- $a \cdot b$ = $\sum _{i=1}^n a_ib_i$  = $a_1b_1 + a_2b_2 + a_3b_3 + a_4b_4 + ... + a_nb_n$


In [28]:
# calculate P(potter|harry)

vector_of_potter = word_vectors[tok2idx['potter']]
vector_of_harry, vector_of_potter

dot_product_value_between_potter_harry = torch.dot(vector_of_harry, vector_of_potter)
dot_product_value_between_potter_harry

tensor(-0.0041)

In [ ]:
# we can get the dot product value for every other words in the vocab
# to get  P(word | harry)
word_dot_dict = {}
for word in filtered_vocab:
  word_vector = word_vectors[tok2idx[word]]
  dot_product_value = torch.dot(vector_of_harry, word_vector)
  word_dot_dict[word] = dot_product_value.item()

word_dot_dict

In [ ]:
from math import exp
# we have to convert our prediction into probability distribution
# P(word|harry)
# sum of [P(a|harry), ..., P(potter|harry), ... P(ron|harry), ... ] = 1

# every probability should be in range (0, 1) (greater than 0, smaller than 1)
# this can be handled by taking exponential of dot product values, divided by total sum
# This function is called Softmax

# Why we use exponential?
# Because we want to make every probability in positive range
# while preserving the order

word_exp_dict = {}
sum_exp_value = 0
for key, value in word_dot_dict.items(): # item is (key, value)
  exp_value = exp(value)
  word_exp_dict[key] = exp_value
  sum_exp_value += exp_value

word_prob_dict = {}
for key, value in word_exp_dict.items(): # item is (key, value)
  word_prob_dict[key] = value / sum_exp_value

word_exp_dict,
# print(sum_exp_value)
word_prob_dict

In [35]:
word_prob_dict['potter'] #P(potter|harry)

0.0005748288528250344

## Matrix Multiplication as Dot Product
![img](https://mkang32.github.io/images/python/khan_academy_matrix_product.png)

In [44]:
# get dot product result for every word in the vocabulary
print(vector_of_harry.shape, word_vectors.shape)

# first, make vector_of_harry into matrix format
mat_of_harry = vector_of_harry.unsqueeze(0)
print(vector_of_harry.shape, mat_of_harry.shape, vector_of_harry, mat_of_harry, )
print(word_vectors.T.shape)

# do matrix multiplication
mm_result = torch.mm(mat_of_harry, word_vectors.T) #mm means matrix multiplication
mm_result.shape # (1 x 8) x (8 x 1701) = (1 x 1701)


torch.Size([8]) torch.Size([1701, 8])
torch.Size([8]) torch.Size([1, 8]) tensor([-0.1886, -0.3664,  0.4148,  0.0823,  0.0359,  0.0883, -0.3901,  0.1089]) tensor([[-0.1886, -0.3664,  0.4148,  0.0823,  0.0359,  0.0883, -0.3901,  0.1089]])
torch.Size([8, 1701])


torch.Size([1, 1701])

In [48]:
word_dot_dict['potter'], mm_result[0, tok2idx['potter']].item()


(-0.004148811101913452, -0.004148811101913452)

In [50]:
# convert dot product result into exponential
exp_mm_result = torch.exp(mm_result)
word_exp_dict['potter'], exp_mm_result[0, tok2idx['potter']].item()


(0.9958597833252065, 0.9958598017692566)

In [54]:
# get the sum of exponential
print(exp_mm_result.shape)
sum_exp_mm = torch.sum(exp_mm_result, dim=1)
sum_exp_mm

torch.Size([1, 1701])


tensor([1732.4457])

In [57]:
# divide exponential value with sum
prob_mm = exp_mm_result / sum_exp_mm

word_prob_dict['potter'], prob_mm[0, tok2idx['potter']].item()

(0.0005748288528250344, 0.0005748288822360337)

In [64]:
def get_probs(query_vectors, entire_vectors):
  assert query_vectors.ndim == 2
  assert entire_vectors.ndim == 2
  assert query_vectors.shape[1] == entire_vectors.shape[1], 'each vector size has to match'

  mm_result = torch.mm(query_vectors, entire_vectors.T)
  exp_value = torch.exp(mm_result)
  sum_value = torch.sum(exp_value, dim=1, keepdim=True)
  print(exp_value.shape, sum_value.shape)
  print(sum_value)
  prob = exp_value / sum_value
  return prob

# get_probs(mat_of_harry, word_vectors)
mat_of_harry_potter = torch.stack([vector_of_harry, vector_of_potter], dim=0)
print(mat_of_harry_potter.shape)
probs = get_probs(mat_of_harry_potter, word_vectors)

torch.sum(probs[1, : ]), torch.sum(probs[0, : ])

torch.Size([2, 8])
torch.Size([2, 1701]) torch.Size([2, 1])
tensor([[1732.4457],
        [1726.2122]])


(tensor(1.), tensor(1.0000))

In [ ]:
word_idx, context_word_idx, len(sample_sentence)